In [1]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

import sklearn
from sklearn.linear_model import LinearRegression

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import time


In [2]:
# check if CUDA is available
use_cuda = torch.cuda.is_available()
print(use_cuda)

False


C:\Users\amallet\Anaconda3\envs\udacity_env\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1)
train_csv = pd.read_csv("train.csv")
print(len(train_csv))
test_csv = pd.read_csv("test.csv")
print(len(test_csv))
samples_submission_csv = pd.read_csv("sample_submission.csv")

1460
1459


In [4]:
print(train_csv)

    Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0    1          60       RL         65.0     8450   Pave   NaN      Reg   
..  ..         ...      ...          ...      ...    ...   ...      ...   

   LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
0          Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
..         ...       ...       ...       ...          ...        ...   

   Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
0        Norm     1Fam     2Story            7            5       2003   
..        ...      ...        ...          ...          ...        ...   

    YearRemodAdd RoofStyle RoofMatl Exterior1st Exterior2nd MasVnrType  \
0           2003     Gable  CompShg     VinylSd     VinylSd    BrkFace   
..           ...       ...      ...         ...         ...        ...   

    MasVnrArea ExterQual ExterCond Foundation BsmtQual BsmtCond BsmtExposure  \
0        196.0        Gd       

In [5]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
train_data= train_data.drop(['SalePrice'], axis='columns')
train_data= train_data.drop(['Id'], axis='columns')
test_data= test_data.drop(['Id'], axis='columns')
all_data = pd.concat([train_data, test_data], keys=["train", "test"])
all_data = pd.get_dummies(all_data)
all_data = all_data.fillna(0)
all_data = (all_data - all_data.mean()) / all_data.std()
print(all_data.shape)

train_data = all_data.loc['train']
print(train_data.shape)
test_data = all_data.loc['test']
print(test_data.shape)

train_data_target = train_csv.iloc[:,-1:]
print(train_data_target.shape)

(2919, 288)
(1460, 288)
(1459, 288)
(1460, 1)


In [6]:
reg = LinearRegression().fit(train_data, train_data_target)
reg.score(train_data, train_data_target)

0.933219970204217

In [7]:
coef = reg.coef_

print(coef.shape)

(1, 288)


In [8]:
reg.intercept_

array([-3.11691077e+13])

In [9]:
output = reg.predict(test_data)
print(len(output))

1459


In [10]:
output


array([[112528.0234375],
       [160121.4609375],
       [186780.5859375],
       ...,
       [176154.4609375],
       [115262.7109375],
       [223324.4296875]])

In [27]:
print(output[0][0])

112528.0234375


In [36]:
import csv

submission = pd.read_csv('sample_submission.csv')
print(submission)

for i in range(len(output)):
    submission.iloc[i][1] = output[i][0]

pd.set_option("display.max_rows", 10, "display.max_columns", None)

submission.to_csv(path_or_buf='sample_submission_SKlearn.csv', index=False)

        Id      SalePrice
0     1461  169277.052498
1     1462  187758.393989
2     1463  183583.683570
3     1464  179317.477511
4     1465  150730.079977
...    ...            ...
1454  2915  167081.220949
1455  2916  164788.778231
1456  2917  219222.423400
1457  2918  184924.279659
1458  2919  187741.866657

[1459 rows x 2 columns]


C:\Users\amallet\Anaconda3\envs\udacity_env\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
